In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time, TimeDelta
from astropy.coordinates import ICRS
import astropy.units as u


location = EarthLocation.of_site('Cerro Pachon')
t0 = Time.now()
delta_t = 30 * u.second
times = [t0, t0 + TimeDelta(30 * u.second)]

# Define azimuth and elevation grid
az_grid = np.linspace(0, 360, 180) * u.deg
el_grid = np.linspace(5, 85, 76) * u.deg

AZ, EL = np.meshgrid(az_grid, el_grid)
az_flat = AZ.ravel()
el_flat = EL.ravel()

altaz0 = AltAz(az=az_flat, alt=el_flat, obstime=t0, location=location)
radec = altaz0.transform_to(ICRS())
altaz1 = radec.transform_to(AltAz(obstime=times[1], location=location))

# Compute elevation change in arcseconds
delta_el = (altaz1.alt - el_flat).to(u.deg).value
delta_el = delta_el.reshape(AZ.shape)

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(8, 8))
c = ax.contourf(
    np.deg2rad(AZ.value),      
    90 - EL.value,              
    delta_el,                   
    levels=50,
    cmap='bwr'
)

cb = fig.colorbar(c, ax=ax, orientation='horizontal', pad=0.1, label='Elevation change over 30s [arcsec]')

ax.set_title("Elevation Change over 30s (Cerro Pachón)", va='bottom', pad=40)
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.set_rlabel_position(135)
#ax.set_rticks([10, 20, 30, 40, 50, 60, 70, 80], 90 - np.array([10, 20, 30, 40, 50, 60, 70, 80]))  # Elevation ticks

plt.tight_layout()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import yaml
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(24, 8))

labels = ['dZ', 'dX', 'dY']
dof_indices = [2, 0, 1]

for ax, label, dof_idx in zip(axs, labels, dof_indices):
    lut_coeffs = lut_data['camera_config']['elevation_coeffs'][dof_idx]
    print(lut_coeffs)
    
    az_grid = np.linspace(0, 360, 180)
    el_grid = np.linspace(5, 85, 76)
    AZ, EL = np.meshgrid(az_grid, el_grid)
    
    # Compute dz at original elevation and elevation + delta_el
    dz_before = np.polyval(lut_coeffs[::-1], EL)  # µm
    dz_after = np.polyval(lut_coeffs[::-1], EL + delta_el)
    dz_diff = np.abs(dz_after - dz_before)  # expected dz offset caused by elevation change
    
    # Plot as a polar heatmap
    az_rad = np.deg2rad(AZ)
    r = 90 - EL
    
    c = ax.contourf(az_rad, r, dz_diff, shading='auto', cmap='viridis',  levels=20)
    cb = fig.colorbar(c, ax=ax, orientation='horizontal', pad=0.1)
    cb.set_label(f'{label} offset [µm]')
    
    ax.set_title(f"Expected Camera {label} LUT Offset from Elevation Change Over 30s", pad=25)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(135)
    ax.set_rticks([10, 30, 50, 70, 85])

    
fig.subplots_adjust(right=0.9, wspace=0.3)
plt.savefig("/home/c/cslage/u/Hexapods/LUTs/Camera_XYZ_Change.png")
#plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import yaml
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(24, 8))

labels = ['dZ', 'dX', 'dY']
dof_indices = [2, 0, 1]

for ax, label, dof_idx in zip(axs, labels, dof_indices):
    lut_coeffs = lut_data['m2_config']['elevation_coeffs'][dof_idx]
    print(lut_coeffs)
    
    az_grid = np.linspace(0, 360, 180)
    el_grid = np.linspace(5, 85, 76)
    AZ, EL = np.meshgrid(az_grid, el_grid)
    
    # Compute dz at original elevation and elevation + delta_el
    dz_before = np.polyval(lut_coeffs[::-1], EL)  # µm
    dz_after = np.polyval(lut_coeffs[::-1], EL + delta_el)
    dz_diff = np.abs(dz_after - dz_before)  # expected dz offset caused by elevation change
    
    # Plot as a polar heatmap
    az_rad = np.deg2rad(AZ)
    r = 90 - EL
    
    c = ax.contourf(az_rad, r, dz_diff, shading='auto', cmap='viridis',  levels=20)
    cb = fig.colorbar(c, ax=ax, orientation='horizontal', pad=0.1)
    cb.set_label(f'{label} offset [µm]')
    
    ax.set_title(f"Expected M2 {label} LUT Offset from Elevation Change Over 30s", pad=25)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(135)
    ax.set_rticks([10, 30, 50, 70, 85])

    
fig.subplots_adjust(right=0.9, wspace=0.3)
plt.savefig("/home/c/cslage/u/Hexapods/LUTs/M2_XYZ_Change.png")
#plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import yaml
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(24, 8))

labels = ['rx', 'ry', 'rz']
dof_indices = [3, 4, 5]

for ax, label, dof_idx in zip(axs, labels, dof_indices):
    lut_coeffs = lut_data['camera_config']['elevation_coeffs'][dof_idx]
    print(lut_coeffs)
    
    az_grid = np.linspace(0, 360, 180)
    el_grid = np.linspace(5, 85, 76)
    AZ, EL = np.meshgrid(az_grid, el_grid)
    
    # Compute dz at original elevation and elevation + delta_el
    dz_before = np.polyval(lut_coeffs[::-1], EL)  # µm
    dz_after = np.polyval(lut_coeffs[::-1], EL + delta_el)
    dz_diff = np.abs(dz_after - dz_before) * 3600.0  # expected dz offset caused by elevation change
    
    # Plot as a polar heatmap
    az_rad = np.deg2rad(AZ)
    r = 90 - EL
    
    c = ax.contourf(az_rad, r, dz_diff, shading='auto', cmap='viridis',  levels=20)
    cb = fig.colorbar(c, ax=ax, orientation='horizontal', pad=0.1)
    cb.set_label(f'{label} offset [arcsec]')
    
    ax.set_title(f"Expected Camera {label} LUT Offset from Elevation Change Over 30s", pad=25)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(135)
    ax.set_rticks([10, 30, 50, 70, 85])

    
fig.subplots_adjust(right=0.9, wspace=0.3)
plt.savefig("/home/c/cslage/u/Hexapods/LUTs/Camera_RXRYRZ_Change.png")
#plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import yaml
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(24, 8))

labels = ['rx', 'ry', 'rz']
dof_indices = [3, 4, 5]

for ax, label, dof_idx in zip(axs, labels, dof_indices):
    lut_coeffs = lut_data['m2_config']['elevation_coeffs'][dof_idx]
    print(lut_coeffs)
    
    az_grid = np.linspace(0, 360, 180)
    el_grid = np.linspace(5, 85, 76)
    AZ, EL = np.meshgrid(az_grid, el_grid)
    
    # Compute dz at original elevation and elevation + delta_el
    dz_before = np.polyval(lut_coeffs[::-1], EL)  # µm
    dz_after = np.polyval(lut_coeffs[::-1], EL + delta_el)
    dz_diff = np.abs(dz_after - dz_before) * 3600.0  # expected dz offset caused by elevation change
    
    # Plot as a polar heatmap
    az_rad = np.deg2rad(AZ)
    r = 90 - EL
    
    c = ax.contourf(az_rad, r, dz_diff, shading='auto', cmap='viridis',  levels=20)
    cb = fig.colorbar(c, ax=ax, orientation='horizontal', pad=0.1)
    cb.set_label(f'{label} offset [arcsec]')
    
    ax.set_title(f"Expected M2 {label} LUT Offset from Elevation Change Over 30s", pad=25)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(135)
    ax.set_rticks([10, 30, 50, 70, 85])

    
fig.subplots_adjust(right=0.9, wspace=0.3)
plt.savefig("/home/c/cslage/u/Hexapods/LUTs/M2_RXRYRZ_Change.png")
#plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import yaml
home_path = os.getenv("HOME")
lut_path_update = os.path.join(
    home_path,
    "notebooks/lsst-ts/ts_config_mttcs/MTHexapod/v5/_init.yaml",
)

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(24, 8))

labels = ['dZ', 'dX', 'dY']
dof_indices = [2, 0, 1]

for ax, label, dof_idx in zip(axs, labels, dof_indices):
    lut_coeffs = lut_data['m2_config']['elevation_coeffs'][dof_idx]

    print(lut_coeffs)
    az_grid = np.linspace(0, 360, 180)
    el_grid = np.linspace(5, 85, 76)
    AZ, EL = np.meshgrid(az_grid, el_grid)
    
    # Compute dz at original elevation and elevation + delta_el
    dz_before = np.polyval(lut_coeffs[::-1], EL)  # µm
    dz_after = np.polyval(lut_coeffs[::-1], EL + delta_el)
    dz_diff = np.abs(dz_after - dz_before)  # expected dz offset caused by elevation change
    
    # Plot as a polar heatmap
    az_rad = np.deg2rad(AZ)
    r = 90 - EL
    
    c = ax.contourf(az_rad, r, dz_diff, shading='auto', cmap='viridis',  levels=20)
    cb = fig.colorbar(c, ax=ax, orientation='horizontal', pad=0.1)
    cb.set_label(f'{label} offset [µm]')
    
    ax.set_title(f"Expected M2 {label} LUT Offset from Elevation Change Over 30s", pad=25)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(135)
    ax.set_rticks([10, 30, 50, 70, 85])

    
fig.subplots_adjust(right=0.9, wspace=0.3)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import yaml
home_path = os.getenv("HOME")
lut_path_update = os.path.join(
    home_path,
    "/home/c/cslage/u/Hexapods/LUTs/_init_25Aug25.yaml",
)

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

fig, axs = plt.subplots(1, 3, subplot_kw={'projection': 'polar'}, figsize=(24, 8))

labels = ['rx', 'ry', 'rz']
dof_indices = [3, 4, 5]

for ax, label, dof_idx in zip(axs, labels, dof_indices):
    lut_coeffs = lut_data['m2_config']['elevation_coeffs'][dof_idx]
    print(lut_coeffs)
    
    az_grid = np.linspace(0, 360, 180)
    el_grid = np.linspace(5, 85, 76)
    AZ, EL = np.meshgrid(az_grid, el_grid)
    
    # Compute dz at original elevation and elevation + delta_el
    dz_before = np.polyval(lut_coeffs[::-1], EL)  # µm
    dz_after = np.polyval(lut_coeffs[::-1], EL + delta_el)
    dz_diff = np.abs(dz_after - dz_before)  # expected dz offset caused by elevation change
    
    # Plot as a polar heatmap
    az_rad = np.deg2rad(AZ)
    r = 90 - EL
    
    c = ax.contourf(az_rad, r, dz_diff, shading='auto', cmap='viridis',  levels=20)
    cb = fig.colorbar(c, ax=ax, orientation='horizontal', pad=0.1)
    cb.set_label(f'{label} offset [deg]')
    
    ax.set_title(f"Expected M2 {label} LUT Offset from Elevation Change Over 30s", pad=25)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(135)
    ax.set_rticks([10, 30, 50, 70, 85])

    
fig.subplots_adjust(right=0.9, wspace=0.3)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import yaml
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

labels = ['dZ', 'dX', 'dY']
dof_indices = [2, 0, 1]

dof_idx = 1
lut_coeffs = lut_data['camera_config']['elevation_coeffs'][dof_idx]
print(lut_coeffs)

el_grid = np.linspace(5, 85, 76)
dy = np.polyval(lut_coeffs[::-1], el_grid)  # µm
dy *= 0.2 / 10 # Convert to arcsec
plt.plot(el_grid, dy)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import yaml
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init_25Aug25.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

labels = ['rx', 'ry', 'rz']
dof_indices = [3, 4, 5]

dof_idx = 4
lut_coeffs = lut_data['m2_config']['elevation_coeffs'][dof_idx]
print(lut_coeffs)

el_grid = np.linspace(5, 85, 76)
dr = np.polyval(lut_coeffs[::-1], el_grid)  # degrees
dr *= 3600# Convert to arcsec
plt.plot(el_grid, dr)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

rot_grid = np.linspace(-90, 90, 181)

import yaml
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init_25Aug25.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

labels = ['rx', 'ry', 'rz']
dof_indices = [3, 4, 5]

dof_idx = 1
lut_coeffs = lut_data['camera_config']['rotation_coeffs'][dof_idx]
print(lut_coeffs)

dr = np.polyval(lut_coeffs[::-1], rot_grid)  # degrees
if dof_idx > 2:
    dr *= 3600# Convert to arcsec
plt.plot(rot_grid, dr)

In [ ]:
lut_data